### QGIS CSV merges per PHU
**Authors:** Sofia and KT

In [24]:
import os
import numpy as np
import pandas as pd
import glob
import missingno as msno

---
### Import and Clean Data


In [25]:
os.chdir('../data/QGIS_csv_files/')

FileNotFoundError: [Errno 2] No such file or directory: '../data/QGIS_csv_files/'

In [ ]:
# read data
prox = pd.read_csv('../PMD-en/PMD-on.csv')
qgis = pd.read_csv('../QGIS_csv_files/Joined_DB_to_PHU.csv')
amenities = pd.read_csv('../amenity_score.csv')
covid = pd.read_csv('../covid_comorbidities.csv')
census = pd.read_excel('../PMD-en/phu_census.xlsx', sheet_name = "ON")

In [ ]:
# only keep necessary columns
qgis = qgis[['fid', 'DBUID', 'PRUID', 'PRNAME', 'CSDUID', 'CSDNAME', 'CSDTYPE', 'CMAUID', 'CMAPUID', 
     'CMANAME', 'CMATYPE', 'HR_UID', 'ENG_LABEL', 'FRE_LABEL']]

In [ ]:
# remove rows that don't have prox data ~ 141 rows removed
prox2 = prox[prox['prox_idx_emp'] != 'F']

# convert .. to np.nan
list_prox = ['prox_idx_emp', 'prox_idx_pharma', 'prox_idx_childcare',
            'prox_idx_health', 'prox_idx_grocery', 'prox_idx_educpri',
            'prox_idx_educsec', 'prox_idx_lib', 'prox_idx_parks',
            'prox_idx_transit']

for each in list_prox:
    prox2[each] = prox2[each].replace({"..":np.nan})

# convert to float
prox2[list_prox] = prox2[list_prox].astype(float)

---
### Merge PHU covid data with PHU census data

In [ ]:
covidmerge = pd.merge(covid, census, how = 'inner', on = 'DGUID')

In [ ]:
# convert census type
covidmerge['Census'] = covidmerge['Census'].str.replace(',', '').astype(int)

---
### Rename QGIS data HRs to PHUs given by Ontario Case Data

> ***Old Comment:*** (Reference `comorbidities-merges.ipynb` for justifications.)

`Location` column in `covidmerge` and `ENG_LABEL` in `qgis` represent PHU.

`Reporting_PHU` has different names.

In [ ]:
# QGIS
qgis.rename(columns = {'ENG_LABEL' : 'Location'}, inplace = True)

# qgis.Reporting_PHU.replace({'York Regional Health Unit' : 'York Region Public Health Services',
#               'Waterloo Health Unit' : 'Region of Waterloo, Public Health',
#             'Halton Regional Health Unit' : 'Halton Region Health Department',
#               'Lambton Health Unit' : 'Lambton Public Health',
#              'Durham Regional Health Unit' : 'Durham Region Health Department',  
#              'City of Ottawa Health Unit' : 'Ottawa Public Health', 
#              'Peel Regional Health Unit': 'Peel Public Health',
#               'Wellington-Dufferin-Guelph Health Unit': 'Wellington-Dufferin-Guelph Public Health',
#             'Kingston, Frontenac and Lennox and Addington Health Unit' : 'Kingston, Frontenac and Lennox & Addington Public Health',
#              'The Eastern Ontario Health Unit':'Eastern Ontario Health Unit',
#               'City of Hamilton Health Unit':'Hamilton Public Health Services',
#               'Niagara Regional Area Health Unit':'Niagara Region Public Health Department',
#               'Sudbury and District Health Unit':'Sudbury & District Health Unit',
#              'The District of Algoma Health Unit':'Algoma Public Health Unit',
#              'Peterborough County - City Health Unit':'Peterborough Public Health',
#              'City of Toronto Health Unit' : 'Toronto Public Health',
#               'Perth District Health Unit' : 'Huron Perth District Health Unit',
#             'Huron County Health Unit' : 'Huron Perth District Health Unit',
#             'Elgin-St. Thomas Health Unit' : 'Southwestern Public Health',
#             'Oxford County Health Unit' : 'Southwestern Public Health'}, inplace = True)

In [ ]:
# merge
merge = pd.merge(qgis, prox2, how = 'inner', on = 'DBUID')

In [ ]:
# convert DBPOP to int
merge['DBPOP'] = merge['DBPOP'].str.replace(",", "")
merge['DBPOP'].replace({np.nan:0}, inplace = True)
merge['DBPOP'] = merge['DBPOP'].map(lambda x: int(x))

In [ ]:
merge[merge['DBPOP'].isna()]

### Missing data by DB

In [ ]:
msno.matrix(prox2.groupby('DBUID').median())

In [ ]:
# calc median of proximity per PHU
phu_med = merge.groupby('Location').median()

In [ ]:
# convert DBPOP from median to sum
phu_med['DBPOP'] = merge.groupby('Location').sum()['DBPOP']

### Merge Amenity_Dense Data (proximity info)

In [ ]:
phu_med = pd.merge(amenities, phu_med, on = "Location", how = 'inner')

In [ ]:
phu_med.columns

#### missing data by PHU

In [ ]:
# missing data by PHU
msno.matrix(phu_med[['prox_idx_transit', 'prox_idx_grocery', 'prox_idx_health', 'prox_idx_parks',]])

In [ ]:
# merge covid with proximity/QGIS data
fullmerge = pd.merge(covidmerge, phu_med, how = 'outer', on = 'Location')



**Calculate proportions and stratified totals for covid numbers:**


DBs were grouped by PHU and sum `DBPOP` (population of PHU). Stratified proportions of COVID cases were calculated by $phu_{weight}*phu_{proportion}$

where 

$phu_{weight} = \frac{phu_{pop}}{ontario_{census}}$

and

$phu_{proportion} = \frac{cases}{phu_{pop}}$.

In [ ]:
convert = ['FEMALE', 'MALE', 'TRANSGENDER', 'UNKNOWN', 'OTHER', 'CONTACT', 'NEITHER', 'TRAVEL-RELATED', 'NOT-RESOLVED',
          'RESOLVED', 'FATAL', 'TOTAL']

for c in convert:
    fullmerge[c+'prop'] = fullmerge[c]/fullmerge['DBPOP']
    

In [ ]:
fullmerge['phu_weight'] = fullmerge['DBPOP']/fullmerge['Census']
fullmerge['TotalStratified'] = fullmerge['phu_weight']*fullmerge['TOTALprop']

fullmerge['FatalStratified'] = fullmerge['phu_weight']*fullmerge['FATALprop']

In [ ]:
# numeric columns to keep 
numeric_keep = ['DGUID', 'DBPOP', 'Reporting_PHU', 'copd', 'asthma', 'hbp', 'smokers', 'FEMALE', 'MALE', 'TRANSGENDER', 
                'UNKNOWN', 'OTHER', 'CONTACT', 'NEITHER', 'TRAVEL-RELATED', 'NOT-RESOLVED', 'RESOLVED', 'FATAL',
               'TOTAL', 'Census', 'Location', 'fid', 'DBUID', 'PRUID_x', 'CSDUID_x', 'CMAUID_x', 'CMAPUID_x', 'HR_UID', 'DAUID',
               'lon', 'lat', 'prox_idx_emp', 'prox_idx_pharma', 'prox_idx_childcare', 'prox_idx_health', 
                'prox_idx_grocery', 'prox_idx_educpri', 'prox_idx_educsec', 'prox_idx_lib', 'prox_idx_parks', 
                'prox_idx_transit', 'amenity']

# proportion columns to keep
prop_keep = ['DGUID', 'DBPOP', 'Reporting_PHU', 'copd-percent', 'asthma-percent', 'hbp-percent', 'smokers-percent', 
            'Census', 'phu_weight', 'TotalStratified', 'FatalStratified',
             'Location', 'fid', 'DBUID', 'PRUID_x', 'CSDUID_x', 'CMAUID_x', 
             'CMAPUID_x', 'HR_UID', 'DAUID', 'lon', 'lat', 'prox_idx_emp', 'prox_idx_pharma', 'prox_idx_childcare', 
             'prox_idx_health', 'prox_idx_grocery', 'prox_idx_educpri', 'prox_idx_educsec', 'prox_idx_lib', 'prox_idx_parks', 
                'prox_idx_transit', 'amenity']



In [ ]:
fullmerge[prop_keep].columns

In [ ]:
fullmerge[prop_keep].to_csv('../PMD-en/PHU_FINAL_prop.csv')
fullmerge[numeric_keep].to_csv('../PMD-en/PHU_FINAL_num.csv')